In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib import colormaps
import warnings
warnings.filterwarnings("ignore")

from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset, plot_trigger
from obspy import Trace

import os
import csv
import pandas as pd

from collections import defaultdict

from geopy.distance import geodesic

from datetime import datetime, timedelta, timezone

from tqdm.auto import tqdm
from icecream import ic

from picking.P_picking import P_picking 
from picking.p_wave_detection import p_picking
from picking.utils import optimize_parameters, load_best_params

from itertools import product
import itertools

import plotly.express as px
import plotly.graph_objects as go

import pickle

In [2]:
# entrega de vuelta un diccionario con la extensión del archivo como llave, y el path a cada archivo como valor
def find_files(path, extensions):
    file_list = {ext: [] for ext in list(extensions)}
    for root, dirs, files in os.walk(path):
        for file in files:
            for ext in extensions:
                if file.endswith(ext):
                    file_list[ext].append(os.path.join(root, file))

    return file_list

# Junta los archivos por carpeta y canal. Queda un diccionario que tiene como llave la carpeta y el canal, 
# y como valor una lista con los paths a los archivos de esa carpeta y canal.
def sort_files(file_dic, extension):
    # Diccionario para almacenar los archivos por carpeta y canal
    grouped_files = defaultdict(list)
    key_names = []
    for file in file_dic[extension]:
        # Extraer la carpeta y el canal del path del archivo
        parts = file.split("\\")
        folder = parts[-2]  # La carpeta es el penúltimo elemento en el path
        channel = parts[-1].split('_')[0]  # El canal es el primer elemento en el nombre del archivo
        key = folder + "\\" + channel
        if [key] not in key_names:
            key_names.append([key])
        # Agrupar los archivos
        grouped_files[key].append(file)

    key_names = list(itertools.chain(*key_names))
    return grouped_files, key_names


In [3]:
files_bhz = find_files('señales_sismos\BHZ', ['.mseed'])
files_bhz_ch, key_names_bhz = sort_files(files_bhz, '.mseed')

In [4]:
#estación principal CO10
main_st = files_bhz_ch[key_names_bhz[3]]
st_main = read(main_st[0])
st_main += read(main_st[1])
st_main += read(main_st[2])
st_main.filter('bandpass', freqmin=4.0, freqmax=10.0) 
print(st_main.select(channel='BHZ'))

#estación adyacente 1: AC04
adj_st_1 = files_bhz_ch[key_names_bhz[0]]
st_adj_1 = read(adj_st_1[0])
st_adj_1 += read(adj_st_1[1])
st_adj_1 += read(adj_st_1[2])
st_adj_1.filter('bandpass', freqmin=4.0, freqmax=10.0)
print(st_adj_1.select(channel='BHZ'))

#estación adyacente 2: AC05
adj_st_2 = files_bhz_ch[key_names_bhz[1]]
st_adj_2 = read(adj_st_2[0])
st_adj_2 += read(adj_st_2[1])
st_adj_2 += read(adj_st_2[2])
st_adj_2.filter('bandpass', freqmin=4.0, freqmax=10.0)
print(st_adj_2.select(channel='BHZ'))

#estación adyacente 3: CO05
adj_st_3 = files_bhz_ch[key_names_bhz[2]]
st_adj_3 = read(adj_st_3[0])
st_adj_3 += read(adj_st_3[1])
st_adj_3 += read(adj_st_3[2])
st_adj_3.filter('bandpass', freqmin=4.0, freqmax=10.0)
print(st_adj_3.select(channel='BHZ'))

1 Trace(s) in Stream:
C1.CO10..BHZ | 2021-07-04T00:00:00.000000Z - 2021-07-05T00:00:00.000000Z | 40.0 Hz, 3456001 samples
1 Trace(s) in Stream:
C1.AC04..BHZ | 2021-07-04T00:00:00.000000Z - 2021-07-05T00:00:00.000000Z | 40.0 Hz, 3456001 samples
1 Trace(s) in Stream:
C1.AC05..BHZ | 2021-07-04T00:00:00.000000Z - 2021-07-05T00:00:00.000000Z | 40.0 Hz, 3456001 samples
1 Trace(s) in Stream:
C1.CO05..BHZ | 2021-07-04T00:00:00.000000Z - 2021-07-05T00:00:00.000000Z | 40.0 Hz, 3456001 samples


In [5]:
# Que las estaciones partan en la hora 01:00:00 y terminen en la hora 04:30:00
st_main = st_main.slice(starttime=UTCDateTime(2021, 7, 4, 1, 0, 0), endtime=UTCDateTime(2021, 7, 4, 4, 30, 0))
st_adj_1 = st_adj_1.slice(starttime=UTCDateTime(2021, 7, 4, 1, 0, 0), endtime=UTCDateTime(2021, 7, 4, 4, 30, 0))
st_adj_2 = st_adj_2.slice(starttime=UTCDateTime(2021, 7, 4, 1, 0, 0), endtime=UTCDateTime(2021, 7, 4, 4, 30, 0))
st_adj_3 = st_adj_3.slice(starttime=UTCDateTime(2021, 7, 4, 1, 0, 0), endtime=UTCDateTime(2021, 7, 4, 4, 30, 0))

In [6]:
st_main.select(channel='BHZ')

1 Trace(s) in Stream:
C1.CO10..BHZ | 2021-07-04T01:00:00.000000Z - 2021-07-04T04:30:00.000000Z | 40.0 Hz, 504001 samples

In [7]:
coord_C010 = (-29.24, -71.46)
coord_AC04 = (-28.20, -71.07)
coord_AC05 = (-28.84, -70.27)
coord_CO05 = (-29.92, -71.24)

#Lista de estaciones
stations = [st_main, st_adj_1, st_adj_2, st_adj_3]

#Lista de Coordenadas
coord_list = [coord_C010, coord_AC04, coord_AC05, coord_CO05]

#Valores en segundos de las ventanas de tiempo en que se mueven las ventanas
ventana_10s = 10
ventana_30s = 30

#Velocidad de propagación de la onda P
v_P = 8.046 

In [9]:

# rango de parámetros a probar
nsta_values = [1.5, 1.6, 1.8, 1.9, 2, 2.2, 2.3, 2,5, 2,7]
nlta_values = [8, 8.5, 9, 9.5, 10, 10.5, 11, 12, 12.4, 12.6, 12.8]
thr_on_values = [3, 3.2, 3.6, 3.7, 3.8, 4, 4.2, 4.5, 4.8, 5, 5.5, 6]
thr_off = 3 # solo sirve al momento de hacer plot_trigger, por lo que no es necesario cambiarlo para buscar eventos

best_params, worst_params = optimize_parameters(nsta_values, nlta_values, thr_on_values, thr_off, stations, ventana_10s, ventana_30s, v_P, coord_list)


  0%|          | 0/1452 [00:00<?, ?it/s]

ic| best_params: {(1.5, 8, 6, 3): 0}
ic| best_params: {(1.5, 8, 5.5, 3): 0, (1.5, 8, 6, 3): 0}
ic| best_params: {(1.5, 8, 5, 3): 0.5, (1.5, 8, 5.5, 3): 0, (1.5, 8, 6, 3): 0}
ic| best_params: {(1.5, 8, 4.8, 3): 0.4727272727272727,
                  (1.5, 8, 5, 3): 0.5,
                  (1.5, 8, 5.5, 3): 0,
                  (1.5, 8, 6, 3): 0}
ic| best_params: {(1.5, 8, 4.5, 3): 0.44444444444444453,
                  (1.5, 8, 4.8, 3): 0.4727272727272727,
                  (1.5, 8, 5, 3): 0.5,
                  (1.5, 8, 5.5, 3): 0,
                  (1.5, 8, 6, 3): 0}
ic| best_params: {(1.5, 8, 4.2, 3): 0.3595505617977528,
                  (1.5, 8, 4.5, 3): 0.44444444444444453,
                  (1.5, 8, 4.8, 3): 0.4727272727272727,
                  (1.5, 8, 5, 3): 0.5,
                  (1.5, 8, 5.5, 3): 0}
ic| best_params: {(1.5, 8, 4, 3): 0.3300970873786408,
                  (1.5, 8, 4.2, 3): 0.3595505617977528,
                  (1.5, 8, 4.5, 3): 0.44444444444444453,
             

In [10]:
best_params =(1.5, 12.4, 6.5, 3)

In [10]:
with open('times_events_24hrs_chiquito.txt', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Saltar la cabecera
    tiempos_reales = [datetime.strptime(row[0], '%Y-%m-%dT%H:%M:%S') for row in reader]
    

In [11]:
len(tiempos_reales)

17

In [12]:
# Parámetros óptimos encontrados
nsta, nlta, thr_on, thr_off = (2.2, 10, 4.5, 3) # hacer nsta más grande, hace más chico el ratio. Hacer más grande nlta hace más grande el ratio
time_all = p_picking(stations, ventana_10s, ventana_30s, nsta, nlta, v_P, coord_list, thr_on, thr_off)

In [13]:
with open('time_trigger_chiquito.txt', 'w') as archivo:
    archivo.write('\n'.join(str(time) for time in time_all))

In [14]:
# Función para leer tiempos de un archivo
def read_times(filename, time_format, skip_header=True):
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        if skip_header:
            next(reader)  # Saltar la cabecera
        return [datetime.strptime(row[0], time_format) for row in reader]

# Leer tiempos reales y predichos
tiempos_reales = read_times('times_events_24hrs_chiquito.txt', '%Y-%m-%dT%H:%M:%S')
tiempos_predichos = read_times('time_trigger_chiquito.txt', '%Y-%m-%dT%H:%M:%S.%fZ', skip_header=False)

# Crear conjuntos con tiempos reales y predichos sin segundos
conjunto_reales = set([t.replace(second=0) for t in tiempos_reales])
conjunto_predichos = set([t.replace(second=0, microsecond=0) for t in tiempos_predichos])

# Calcular verdaderos positivos, falsos positivos y falsos negativos
verdaderos_positivos = conjunto_reales & conjunto_predichos
falsos_positivos = conjunto_predichos - conjunto_reales
falsos_negativos = conjunto_reales - conjunto_predichos

resultados = {'Verdaderos Positivos': len(verdaderos_positivos),
              'Falsos Positivos': len(falsos_positivos),
              'Falsos Negativos': len(falsos_negativos)}

# Crear diccionarios para mapear cada tiempo a su segundo correspondiente
dict_tiempos_reales = {t.replace(second=0): t.second for t in tiempos_reales}
dict_tiempos_predichos = {t.replace(second=0, microsecond=0): t.second for t in tiempos_predichos}

# Esto siguiente es para poder tener los tiempos con los segundos, y no solo las horas y minutos
v_positivos = sorted([t.replace(second=dict_tiempos_predichos[t]) for t in verdaderos_positivos])
f_positivos = sorted([t.replace(second=dict_tiempos_predichos[t]) for t in falsos_positivos])
f_negativos = sorted([t.replace(second=dict_tiempos_reales[t]) for t in falsos_negativos])


# Leer magnitudes de un archivo
with open('times_events_24hrs_chiquito.txt', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Saltar la cabecera
    magnitudes = {datetime.strptime(row[0], '%Y-%m-%dT%H:%M:%S'): float(row[1]) for row in reader}

# Mapear cada verdadero positivo a su magnitud correspondiente
# Mapear cada verdadero positivo a su magnitud correspondiente usando los tiempos reales
t_real_v_positivos = sorted([datetime(t.year, t.month, t.day, t.hour, t.minute, dict_tiempos_reales[t]) for t in verdaderos_positivos])
t_real_v_positivos = [datetime.strptime(t.isoformat(), '%Y-%m-%dT%H:%M:%S') for t in t_real_v_positivos]
magnitudes_verdaderos_positivos = {t: magnitudes[t] for t in t_real_v_positivos}

In [24]:
# Leer magnitudes
magnitudes_real = []
with open('times_events_24hrs.txt', 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        magnitudes_real.append(float(row['Magnitud']))

# Utilizar una paleta de colores
cmap = sns.color_palette("viridis", as_cmap=True)

# Crear un objeto de figura de Plotly
fig_magnitud = go.Figure()

# Agregar la traza de dispersión al objeto de figura
fig_magnitud.add_trace(go.Scatter(
    x=tiempos_reales,
    y=magnitudes_real,
    mode='markers',
    name='Magnitudes',
    marker=dict(
        color=magnitudes_real,
        colorscale='Jet',
        size=10,
        colorbar=dict(
            title='Magnitud',
        )
    )
))

# Personalizar diseño
fig_magnitud.update_layout(
    title='Magnitudes de los eventos sísmicos catalogados en tiempo determinado superiores a 3.5',
    xaxis_title='Tiempo Real',
    xaxis=dict(
        tickformat='%Y-%m-%d %H:%M:%S',
        tickangle=-45,
        type='date'
    ),
    yaxis_title='Magnitud',
    font=dict(family="Arial", size=12, color="white"),
    height=600,  # Ajustar la altura del gráfico
    width=1300,    # Ajustar el ancho del gráfico
    paper_bgcolor='dimgray',      # Cambiar el color de fondo del gráfico
    plot_bgcolor='dimgray',       # Cambiar el color de fondo de la trama
    title_font=dict(size=20, color="white")  # Ajustar el tamaño y color del título
)

# Mostrar gráfico interactivo
fig_magnitud.show()

In [18]:
# Crear DataFrame de resultados para Plotly Express
df_resultados = pd.DataFrame({
    'Categorías': ['Verdaderos Positivos', 'Falsos Positivos', 'Falsos Negativos'],
    'Cantidad': [len(verdaderos_positivos), len(falsos_positivos), len(falsos_negativos)]
})

# Configurar el estilo de los gráficos (opcional)
px.defaults.template = "seaborn"

# Graficar los resultados con colores variados usando Plotly Express
fig_resultados = px.bar(df_resultados, x='Categorías', y='Cantidad', color='Categorías')

# Personalizar diseño
fig_resultados.update_layout(
    title='Resultados',
    xaxis_title='Categorías',
    yaxis_title='Cantidad',
    paper_bgcolor='dimgray',      # Cambiar el color de fondo del gráfico
    plot_bgcolor='dimgray',       # Cambiar el color de fondo de la trama
    title_font=dict(size=20, color="white"),  # Ajustar el tamaño y color del título
    font=dict(family="Arial", size=12, color="white")
)

# Reducir el tamaño del gráfico de resultados
fig_resultados.update_layout(height=500, width=1000)

# Mostrar gráfico interactivo
fig_resultados.show()

# Crear DataFrame de magnitudes para Plotly Express
df_magnitudes = pd.DataFrame({
    'Tiempo': list(magnitudes_verdaderos_positivos.keys()),
    'Magnitud': list(magnitudes_verdaderos_positivos.values())
})

# Configurar el estilo de los gráficos (opcional)
px.defaults.template = "seaborn"

# Crear un objeto de figura
fig_magnitud = go.Figure()

# Agregar la traza de línea al objeto de figura

fig_magnitud.add_trace(go.Scatter(
    x=[t.strftime('%Y-%m-%d %H:%M:%S') for t in magnitudes_verdaderos_positivos.keys()],
    y=list(magnitudes_verdaderos_positivos.values()),
    mode='markers',
    name='Magnitudes',
    marker=dict(
        color=list(magnitudes_verdaderos_positivos.values()),  # Asignar la magnitud como la escala de color
        colorscale='Jet',  # Escala de colores
        size = 7,
        colorbar=dict(
            title='Magnitud',  # Etiqueta de la barra de colores
        )
    )
))

# Personalizar diseño
fig_magnitud.update_layout(
    title='Magnitudes de los Verdaderos Positivos',
    xaxis_title='Tiempo',
    xaxis=dict(
        tickformat='%Y-%m-%d %H:%M:%S',
        tickangle=-45,
        type='date'
    ),
    yaxis_title='Magnitud',
    font=dict(family="Arial", size=12, color="white"),
    height=600,  # Ajustar la altura del gráfico
    width=1300,    # Ajustar el ancho del gráfico
    paper_bgcolor='dimgray',      # Cambiar el color de fondo del gráfico
    plot_bgcolor='dimgray',       # Cambiar el color de fondo de la trama
    title_font=dict(size=20, color="white")  # Ajustar el tamaño y color del título
)

# Reducir el tamaño del gráfico de magnitudes
fig_magnitud.update_layout(height=400, width=1000)

# Mostrar gráfico interactivo
fig_magnitud.show()

In [19]:

magnitudes_list = list(magnitudes.values())

df = pd.DataFrame({'Magnitudes': magnitudes_list})

# Crear bins para la data 
bins = np.arange(min(magnitudes_list), max(magnitudes_list) + 0.1, 0.1)
df['Bins'] = pd.cut(df['Magnitudes'], bins=bins, include_lowest=True)
df_bins = df['Bins'].value_counts().sort_index().reset_index()
df_bins.columns = ['Bins', 'Count']
df_bins['Mid'] = df_bins['Bins'].apply(lambda x: (x.right + x.left) / 2)

color_scale = cm.get_cmap('seismic', len(df_bins))
colors = ['rgb'+str(color_scale(i)[:3]) for i in range(len(df_bins))]

# bar chart
fig = go.Figure(data=[go.Bar(x=df_bins['Mid'], y=df_bins['Count'], marker_color=colors, width=0.1)])


fig.update_layout(
    title_text='Cantidad de Sismos (VP) Según Magnitud', # title of plot
    xaxis_title_text='Magnitud', # xaxis label
    yaxis_title_text='Cantidad', # yaxis label
    template="seaborn",
    paper_bgcolor='dimgray',
    plot_bgcolor='dimgray',
    font=dict(family="Arial", size=12, color="white")
)

fig.show()


